In [19]:
import copy
import kubernetes
import common
import yaml
import json


def get_resource_helper(func):
    k8s_namespace = "default"
    response = func(k8s_namespace, _preload_content=False, watch=False)
    data = json.loads(response.data)
    return [resource for resource in data['items']]

def generate_resources():
    print("Generating cluster resources digest ...")
    kubernetes.config.load_kube_config()
    core_v1 = kubernetes.client.CoreV1Api()
    apps_v1 = kubernetes.client.AppsV1Api()
    resources = {}
    
    resources[common.POD] = get_resource_helper(core_v1.list_namespaced_pod)
    resources[common.PVC] = get_resource_helper(core_v1.list_namespaced_persistent_volume_claim)
    resources[common.DEPLOYMENT] = get_resource_helper(apps_v1.list_namespaced_deployment)
    resources[common.STS] = get_resource_helper(apps_v1.list_namespaced_stateful_set)

    return resources

def generate_resources_old():
    print("Generating cluster resources digest ...")
    kubernetes.config.load_kube_config()
    core_v1 = kubernetes.client.CoreV1Api()
    apps_v1 = kubernetes.client.AppsV1Api()
    k8s_namespace = "default"
    resources = {}
    for ktype in common.KTYPES:
        resources[ktype] = []
    for pod in core_v1.list_namespaced_pod(k8s_namespace, watch=False).items:
        resources[common.POD].append(pod)
    for pvc in core_v1.list_namespaced_persistent_volume_claim(k8s_namespace, watch=False).items:
        resources[common.PVC].append(pvc)
    for dp in apps_v1.list_namespaced_deployment(k8s_namespace, watch=False).items:
        resources[common.DEPLOYMENT].append(dp)
    for sts in apps_v1.list_namespaced_stateful_set(k8s_namespace, watch=False).items:
        resources[common.STS].append(sts)
    return resources

In [46]:
kubernetes.config.load_kube_config()
core_v1 = kubernetes.client.CoreV1Api()
apps_v1 = kubernetes.client.AppsV1Api()

pods = core_v1.list_namespaced_pod("default", watch=False)
# print(pods.items)

core_v1.api_client.deserialize
apps_v1.get_api_resources()

{'api_version': 'v1',
 'group_version': 'apps/v1',
 'kind': 'APIResourceList',
 'resources': [{'categories': None,
                'group': None,
                'kind': 'ControllerRevision',
                'name': 'controllerrevisions',
                'namespaced': True,
                'short_names': None,
                'singular_name': '',
                'storage_version_hash': '85nkx63pcBU=',
                'verbs': ['create',
                          'delete',
                          'deletecollection',
                          'get',
                          'list',
                          'patch',
                          'update',
                          'watch'],
                'version': None},
               {'categories': ['all'],
                'group': None,
                'kind': 'DaemonSet',
                'name': 'daemonsets',
                'namespaced': True,
                'short_names': ['ds'],
                'singular_name': '',
            

In [21]:
learn2_old = generate_resources_old()


Generating cluster resources digest ...


In [22]:
test2_old = generate_resources_old()

Generating cluster resources digest ...


In [23]:
from deepdiff import DeepDiff

DeepDiff(learn2_old, test2_old)

{}

In [62]:
learn2 = generate_resources()
learn2
open("learn2.json", "w").write(json.dumps(learn2))



Generating cluster resources digest ...


80137

In [64]:
test2 = generate_resources()
test2
open("test2.json", "w").write(json.dumps(test2))



Generating cluster resources digest ...


62369

In [3]:
from types import SimpleNamespace
learn = json.loads(open("learn1.json").read(), object_hook=lambda d: SimpleNamespace(**d))
test = json.loads(open("test1.json").read(), object_hook=lambda d: SimpleNamespace(**d))

learn = json.loads(open("learn1.json").read())
test = json.loads(open("test1.json").read())

In [43]:
import jsondiff as jd
import copy


not_care_keys = ['uid', 'resourceVersion', 'creationTimestamp', 'ownerReferences', 'managedFields', 'generateName', 'selfLink', 'annotations',
                'pod-template-hash', 'secretName', 'image', 'lastTransitionTime', 'nodeName', 'podIPs', 'hostIP', 'containerID', 'imageID',
                'startTime',' startedAt', 'volumeMounts', 'finishedAt', 'volumeName', 'lastUpdateTime']
not_care = [jd.delete, jd.insert] + not_care_keys + ['name']

def trim_resource(cur):
    if type(cur) is list:
        for item in cur:
            trim_resource(item)
            
    if type(cur) is dict:
        for key in list(cur):
            if key in not_care_keys:
                cur.pop(key, None)
            else:
                trim_resource(cur[key])
                


def look_for_resouces_diff(learn, test):
    learn_trim = copy.deepcopy(learn)
    test_trim = copy.deepcopy(test)
    trim_resource(learn_trim)
    trim_resource(test_trim)
    diff = jd.diff(learn_trim, test_trim, syntax='symmetric')
    
    print(learn_trim)
    
    for rtype in diff:
        print("> diff for", rtype)
        rdiff = diff[rtype]
        # Check for resource delete/insert
        if jd.delete in rdiff:
            # Any resource deleted
            for (idx, item) in rdiff[jd.delete]:
                print("[resource deleted]", item['metadata']['namespace'], item['metadata']['name'], "old status:", item['status']['phase'])


        if jd.insert in rdiff:
            # Any resource added
            for (idx, item) in rdiff[jd.insert]:
                print("[resource added]", item['metadata']['namespace'], item['metadata']['name'], "new status:", item['status']['phase'])

        # Check for resource internal fields
        for idx in rdiff:
            if not type(idx) is int:
                continue
            resource = test[rtype][idx]
            name = resource['metadata']['name']
            namespace = resource['metadata']['namespace']
            item = rdiff[idx]
            # Handle for metadata
            if 'metadata' in item:
                metadata = item['metadata']
                # Also should handle add/delete here
                for key in metadata:
                    if not key in not_care:
                        print("[metadata field changed]", name, key, "changed", "delta: ", metadata[key])
                if jd.delete in metadata:
                    print("[metadata field deleted]", name, metadata[jd.delete])
                if jd.insert in metadata:
                    print("[metadata field added]", name, metadata[jd.insert])

    #             if 'status' in item:
    #                 # Status changed
    #                 print("status changed", item['status'])
        import pprint
        pp = pprint.PrettyPrinter()

        pp.pprint(rdiff)

    
look_for_resouces_diff(learn, test)

{'pod': [{'metadata': {'name': 'cassandra-operator-868569994f-g49w4', 'namespace': 'default', 'labels': {'name': 'cassandra-operator', 'sonartag': 'cassandra-operator'}}, 'spec': {'volumes': [{'name': 'cassandra-operator-token-whdnd', 'secret': {'defaultMode': 420}}], 'containers': [{'name': 'cassandra-operator', 'command': ['./cassandra-operator'], 'envFrom': [{'configMapRef': {'name': 'sonar-testing-global-config'}}], 'env': [{'name': 'WATCH_NAMESPACE', 'valueFrom': {'fieldRef': {'apiVersion': 'v1', 'fieldPath': 'metadata.namespace'}}}, {'name': 'POD_NAME', 'valueFrom': {'fieldRef': {'apiVersion': 'v1', 'fieldPath': 'metadata.name'}}}, {'name': 'OPERATOR_NAME', 'value': 'cassandra-operator'}, {'name': 'KUBERNETES_SERVICE_HOST', 'value': 'kind-control-plane'}, {'name': 'KUBERNETES_SERVICE_PORT', 'value': '6443'}], 'resources': {}, 'terminationMessagePath': '/dev/termination-log', 'terminationMessagePolicy': 'File', 'imagePullPolicy': 'IfNotPresent'}], 'restartPolicy': 'Always', 'termi

In [70]:
from deepdiff import DeepDiff

diff = DeepDiff(learn, test, verbose_level=2)

import pprint
pp = pprint.PrettyPrinter(indent=2)

def output(key):
    print(key)
    pp.pprint(diff[key])
    print()

# output("type_changes")

# output("dictionary_item_added")
# output("dictionary_item_removed")
# output("values_changed")

# print(diff["dictionary_item_added"][0])
diff

{'dictionary_item_added': {'root[\'pod\'][0][\'metadata\'][\'managedFields\'][0][\'fieldsV1\'][\'f:metadata\'][\'f:ownerReferences\'][\'k:{"uid":"7b0e5c31-fa50-4237-88f0-18c0da2f20f1"}\']': {'.': {},
   'f:apiVersion': {},
   'f:blockOwnerDeletion': {},
   'f:controller': {},
   'f:kind': {},
   'f:name': {},
   'f:uid': {}},
  "root['pod'][0]['metadata']['managedFields'][1]['fieldsV1']['f:status']['f:containerStatuses']": {},
  "root['pod'][0]['metadata']['managedFields'][1]['fieldsV1']['f:status']['f:hostIP']": {},
  "root['pod'][0]['metadata']['managedFields'][1]['fieldsV1']['f:status']['f:phase']": {},
  "root['pod'][0]['metadata']['managedFields'][1]['fieldsV1']['f:status']['f:podIP']": {},
  "root['pod'][0]['metadata']['managedFields'][1]['fieldsV1']['f:status']['f:podIPs']": {'.': {},
   'k:{"ip":"10.244.3.7"}': {'.': {}, 'f:ip': {}}},
  "root['pod'][0]['metadata']['managedFields'][1]['fieldsV1']['f:status']['f:startTime']": {},
  'root[\'pod\'][0][\'metadata\'][\'managedFields\

In [61]:
diff['dictionary_item_removed'].keys()

dict_keys(['root[\'pod\'][0][\'metadata\'][\'managedFields\'][0][\'fieldsV1\'][\'f:metadata\'][\'f:ownerReferences\'][\'k:{"uid":"88e232ec-26b2-43c2-a853-af6e0b428a4b"}\']', "root['pod'][0]['metadata']['managedFields'][1]['fieldsV1']['f:status']['f:conditions']['.']", 'root[\'pod\'][0][\'metadata\'][\'managedFields\'][1][\'fieldsV1\'][\'f:status\'][\'f:conditions\'][\'k:{"type":"PodScheduled"}\']', 'root[\'pod\'][1][\'metadata\'][\'managedFields\'][0][\'fieldsV1\'][\'f:metadata\'][\'f:ownerReferences\'][\'k:{"uid":"e0210866-b912-4033-9294-d374980295e1"}\']', 'root[\'pod\'][1][\'metadata\'][\'managedFields\'][1][\'fieldsV1\'][\'f:status\'][\'f:podIPs\'][\'k:{"ip":"10.244.4.6"}\']', "root['deployment'][0]['metadata']['annotations']['kubectl.kubernetes.io/last-applied-configuration']", "root['deployment'][0]['metadata']['managedFields'][0]['fieldsV1']['f:metadata']", 'root[\'statefulset\'][0][\'metadata\'][\'managedFields\'][0][\'fieldsV1\'][\'f:metadata\'][\'f:ownerReferences\'][\'k:{"ui